datagen에서 만든 파일을 가지고 Qwen 모델을 파인튜닝

In [1]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[xformers|WARNING]WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.9.1+cu128)
    Python  3.11.9 (you have 3.11.14)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.9.1+cu128)
    Python  3.11.9 (you have 3.11.14)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not find Config class in trl.trainer.dpo_trainer. Found: []
Unsloth: Could not find Config class in trl.trainer.iterative_sft_trainer. Found: []
Unsloth: Could not find Config class in trl.trainer.sft_trainer. Found: []


모델 로드 및 LoRA 설정

In [2]:
# 모델 설정
model_name = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"
max_seq_length = 2048 # 문맥 길이
dtype = None # None으로 두면 자동 설정 (Float16 등)
load_in_4bit = True # 4bit 양자화로 메모리 절약 [cite: 31]

# 모델 다운로드 및 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA GeForce GTX 1650. Num GPUs = 1. Max memory: 3.596 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
# LoRA 설정
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


데이터셋 로드 및 포맷 변환

In [4]:
# 데이터셋 로드
dataset = load_dataset("json", data_files="6g_ai_dataset_augmented.jsonl", split="train")

In [5]:
# 채팅 템플릿 적용 함수 (Unsloth 표준 방식)
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    
    for instruction, output in zip(instructions, outputs):
        # Qwen 채팅 템플릿 형식으로 변환 (User -> Assistant 구조)
        messages = [
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": output}
        ]
        # 토크나이저가 자동으로 포맷팅 (EOS 토큰 등 추가)
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
        
    return {"text": texts}

In [6]:
# 변환 적용
train_dataset = dataset.map(formatting_prompts_func, batched=True)
print(train_dataset[0]["text"])

<|im_start|>user
2025 6G White Paper의 주요 주제는 무엇입니까?<|im_end|>
<|im_start|>assistant
<think>

</think>

2025 6G White Paper는 6G와 생성형 AI의 결합, 특히 미래 지능을 공동으로 창조하는 방법에 중점을 둡니다.<|im_end|>



학습 시작

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2, # 배치 사이즈 
        gradient_accumulation_steps = 4, # 그래디언트 누적 
        warmup_steps = 5,
        max_steps = 60, # 학습 반복 횟수 (데이터 양에 따라 늘리세요) 
        learning_rate = 2e-4, # 학습률 
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # 8bit 옵티마이저로 메모리 절약 
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [8]:
# 학습 실행
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 36 | Num Epochs = 12 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)


Step,Training Loss
1,3.605200
2,3.159400
3,3.401700
4,3.556700
5,3.069100
6,2.840900
7,2.838100
8,2.567000
9,2.470800
10,2.279300


추론 테스트

In [9]:
from transformers import TextStreamer

# 모델을 추론 모드로 전환
FastLanguageModel.for_inference(model)

# 질문 작성
messages = [
    {"role": "user", "content": "6G 네트워크에서 AI의 역할은 무엇인가요?"} # 여기에 궁금한 질문 입력
]

In [ ]:
# 입력 데이터 처리
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # 답변 생성을 위한 프롬프트 추가 
    return_tensors = "pt",
).to("cuda")

# 답변 생성 및 출력
_ = model.generate(
    input_ids = inputs,
    streamer = TextStreamer(tokenizer, skip_prompt=True),
    max_new_tokens = 256, # 답변 길이 제한 
    use_cache = True,
    temperature = 0.7, 
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<think>

</think>

6G 네트워크와 AI(Advanced Intelligent Technology)의 상호 작용이 주로 AI를 통해 6G와 IoT(Internet of Things) 간의 융합, 5G와 6G 간의 통신 전략(예: 6G-5G Hybrid Edge Internet)을 지원함으로써 6G의 지능형 통신(Advanced Intelligent Communication) 기반 운영을 가능하게 하려는 것으로, AI가 6G를 '지능형 미래 지능'으로 만듭니다.<|im_end|>


모델 저장

In [11]:
save_dir = "outputs/"

# 학습된 모델 + LoRA 가중치 포함 전체 저장
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"모델 저장 완료! 저장 위치: {save_dir}")

모델 저장 완료! 저장 위치: outputs/


Restart 후 저장된 모델 로드해서 테스트

In [3]:
from unsloth import FastLanguageModel

save_dir = "outputs/"

# 저장된 모델 로드
loaded_model, loaded_tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_dir,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(loaded_model)

messages = [
    {"role": "user", "content": "6G 네트워크에서 AI의 역할은 무엇인가요?"}
]

inputs = loaded_tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

output = loaded_model.generate(
    input_ids=inputs,
    max_new_tokens=256,
)

print(loaded_tokenizer.decode(output[0], skip_special_tokens=True))

==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA GeForce GTX 1650. Num GPUs = 1. Max memory: 3.596 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
user
6G 네트워크에서 AI의 역할은 무엇인가요?
assistant
<think>

</think>

미래 6G와 AI가 결합하여 6G 네트워크에서 AI의 핵심 역할은 무엇인지에 대해 다음과 같은 포인트를 제시합니다.

1. **AI를 통합된 6G-와 AI 협력(6G & AI Integration):**  
   AI 기반의 인력(Force) 분석과 데이터(Information) 분석을 결합해 6G 네트워크 운영과 인프라 개선에 기여합니다.  
   (AI를 통합하여 6G와 AI를 결합하는 6G-와 AI 협력(6G & AI Integration)을 강조)

2. **AI를 통해 6G-에 대한 인프라 개선:**  
   AI 기반의 자동화 및 효율성(automation and efficiency) 분석을 통해 6G 네트워크의 운영(Operations)과 성능(Performance)을 빨리게 개선합니다.  
   (AI를 통해 6G-에 대한 인프라 개선에 중점을 둡니다)

3. **AI를 통한 6G
